# Importações Necessárias

In [1]:
import sys, os
import pandas as pd
from tqdm import tqdm
import unidecode
import re
import plotly.express as px

# Funções Auxiliares

In [2]:
def plot_bar(aux_x, aux_y, aux_text, title, x_title, y_title):

    fig = px.bar(x=aux_x, y=aux_y,text=aux_text)

    fig.update_layout(
        title=title,
        xaxis_title=x_title,
        yaxis_title=y_title,
        yaxis=dict(tickformat=',f')
    )

    fig.show()

# Leitura dos Dados

Dataset obtido a partir do repositório https://github.com/roneysco/Fake.br-Corpus

In [3]:
fake_path = os.path.join(os.getcwd(),r'..\Fake.br-Corpus-master\full_texts\fake')
true_path = os.path.join(os.getcwd(),r'..\Fake.br-Corpus-master\full_texts\true')

Módulo que constroi o dataframe de amostras rotuladas e salva na pasta Dados para tornar as leituras seguintes mais rápidas

In [4]:
# df_corpus = pd.DataFrame(columns=['sample','label'])

# for filename in tqdm(os.listdir(fake_path)):
#     f = open(os.path.join(fake_path,filename), encoding="utf8")
#     sample = f.read()

#     df = pd.DataFrame([[sample,1]],columns=['sample','label'])

#     df_corpus = df_corpus.append(df, ignore_index=True)

# for filename in tqdm(os.listdir(true_path)):
#     f = open(os.path.join(true_path,filename), encoding="utf8")
#     sample = f.read()

#     df = pd.DataFrame([[sample,0]],columns=['sample','label'])

#     df_corpus = df_corpus.append(df, ignore_index=True)

# df_corpus.to_pickle('./Dados/raw_corpus.pkl')

Lê o dataframe de amostras rotuladas

In [11]:
df_corpus = pd.read_pickle('./Dados/raw_corpus.pkl')

In [12]:
df_corpus

,sample,label
0,Kátia Abreu diz que vai colocar sua expulsão e...,1
1,"Dr. Ray peita Bolsonaro, chama-o de conservad...",1
2,Reinaldo Azevedo desmascarado pela Polícia Fed...,1
3,Relatório assustador do BNDES mostra dinheiro ...,1
4,"Radialista americano fala sobre o PT: ""Eles ve...",1
...,...,...
7195,"Para jornal britânico, ação contra Lula na Lav...",0
7196,Temer diz que acionou PF e Cade para investiga...,0
7197,Os obstáculos políticos de Temer em 2017. Espe...,0
7198,"Sexta-feira, 15 de setembro de 2017. Boa noite...",0


# Análise dos dados

In [13]:
aux = df_corpus['label'].value_counts().reset_index(name='qtd_rotulos').rename(columns={'index':'rotulo'})
aux['pct_rotulos'] = aux['qtd_rotulos']/aux['qtd_rotulos'].sum()

aux_x = aux['rotulo'].map({1:'falsa',0:'verdadeira'})
aux_y = aux['qtd_rotulos']
aux_text = aux['pct_rotulos'].apply(lambda x: '{:.2f}%'.format(x*100))

plot_bar(aux_x,aux_y,aux_text,'Distribuição dos rótulos no conjunto de dados',
         'rótulo','quantidade de rótulos')

# Pré Processamento

In [14]:
for i in range(600,1000):
    print(i)
    print(df_corpus['sample'][i])

600
O prefeito mais humilde do Brasil  Exemplo pro Brasil. Flagrante do prefeito de uma das cidades mais ricas do Espírito Santo em pleno sábado a tarde (dia de folga) fazendo uma coisa que você nunca viu político nenhum do mundo fazer. Serginho Meneguelli é um político estreante que está levando humanidade para a administração pública. No dia de sua posse ele foi para a cerimônia de bicicleta por morar próximo a prefeitura e acreditar que a mobilidade urbana por veículos individuais além de elevar a poluição ainda aumenta os congestionamentos. No vídeo percebemos o prefeito em pleno sabadão, que para outros prefeitos seria um dia de folga, fazendo um jardim com as próprias mãos para economizar dinheiro público e elevar o nível de amor e zelo da população para com os logradouros públicos.

Ajude a compartilhar este exemplo para inspirar outros prefeitos do Brasil.
601
NASA anuncia descoberta de 7 novos planetas. Um deles é semelhante à Terra e há chances de possuir vida.  Uma equipe i

In [15]:
df_corpus['sample'][0]

'Kátia Abreu diz que vai colocar sua expulsão em uma moldura, mas não para de reclamar.\t\n\nA senadora Kátia Abreu (sem partido-TO) disse que sua expulsão do PMDB foi resultado de uma ação da cúpula atual da legenda que, segundo ela, é oportunista.\n\n“Amanhã eu vou botar numa moldura dourada a minha expulsão, porque das mãos de onde veio, é um atestado de boa conduta para o meu currículo. Essas pessoas que me expulsaram não servem ao país. Eles se servem do país em seus benefícios próprios”, disse Kátia Abreu.\n\nUé, mas se a expulsão é algo tão bom para seu currículo, por que tanta choradeira, Kátia?\n\nSabemos o motivo. Provavelmente Kátia não tem valor para o PT, partido que já deveria tê-la absorvido. Ao que parece o PT gostava de Kátia somente se ela ficasse entrincheirada dentro do PMDB.\n\nOu seja, isso é se rebaixar demais. Resta a Kátia ficar chorando as pitangas por todos os cantos.\n\nEm tempo: até o momento o PT não cadastrou Kátia Abreu em suas fileiras. Que situação pat

Torna todas as letras minúsculas

In [16]:
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: x.lower())

In [17]:
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: unidecode.unidecode(x))

Substitui todos os dígitos por '0'

In [18]:
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: x.replace('"',' ASPAS '))

In [19]:
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: re.sub(r'\d\S+', ' 0 ', x))
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: re.sub(r'\d+', '0', x))

Substitui todos as URLs por 'URL'

In [20]:
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: re.sub(r'http\S+', ' URL ', x))

Substitui todos os e-mails por 'EMAIL'

In [21]:
# df_corpus['sample'] = df_corpus['sample'].apply(lambda x: re.sub(r'\S+@\S+', 'EMAIL', x))

In [22]:
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: re.sub(r'\s@\S+', ' REDESOCIAL ', x))

In [23]:
# df_corpus['sample'] = df_corpus['sample'].apply(lambda x: x.replace('r$',' DINHEIRO '))

In [24]:
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [25]:
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: ' '.join(x.split()))

In [26]:
print(df_corpus['sample'][1366])

site msn aponta o suposto novo affair de fatima bernardes ASPAS ele e mais jovem mas sabe de tudo ASPAS o site msn apontou o suposto novo affair da apresentadora fatima bernardes na noite de ontem 0 fatima postou uma imagem em sua conta no instagram ao lado do produtor rafael tupinamba rafael tambem e funcionario da rede globo e trabalha no programa ASPAS encontro ASPAS ao ladinho de fatima a apresentadora destacou na descricao da imagem que o rapaz e bem mais jovem que ela e escreveu ASPAS ele sabe tudo de tv REDESOCIAL nem era nascido mas pode perguntar pra ele sobre a trilha os atores o desfecho das novelas hj cuida com carinho do elenco que diariamente visita ASPAS encontro ASPAS parceria de sucesso ASPAS escreveu seguidores comecaram a questionar imediatamente ASPAS fatima ja arrumou outro ASPAS questionou um seguidor ASPAS fatima esta certa bola pra frente outra ASPAS disse outro internauta ASPAS casal bonito parabens ASPAS escreveu um terceiro ASPAS linda e vida que segue tchau 

In [28]:
df_corpus['sample'] = df_corpus['sample'].apply(lambda x: x.replace('_',''))

# Tokenização

Tokeniza o texto a partir dos espaços em branco e das pontuações

Salva dataframe pré-processado na pasta Dados

In [29]:
df_corpus.to_pickle('./Dados/preprocessed_corpus.pkl')